# RAG ReAct Loop Demo

This notebook implements a **ReAct (Reasoning + Acting) loop** for RAG queries. Instead of a single retrieval, the system iteratively:

1. **Action**: Query RAG with the current search query
2. **Thought**: LLM evaluates whether the retrieved answer is sufficient
3. **Observation**: If insufficient, LLM generates a refined follow-up query
4. **Repeat** until the answer is complete or max iterations reached
5. **Synthesize**: All accumulated chunks are combined into a comprehensive final answer

### ReAct Loop Flow

```
Original Question
  -> Iteration 1:
       ACTION:      RAG query (original question)
       OBSERVATION: Retrieved chunks + synthesized answer
       THOUGHT:     LLM evaluates: "Is this sufficient?"
         -> Yes: Break, synthesize final answer
         -> No:  Generate follow-up query
  -> Iteration 2:
       ACTION:      RAG query (refined follow-up query)
       OBSERVATION: New chunks accumulated
       THOUGHT:     LLM evaluates again (with memory of previous iterations)
         -> ...
  -> Final Synthesis:
       All accumulated chunks -> LLM -> Comprehensive answer
```

### Prerequisites

1. Both servers must be running (`python tools_server.py` then `python run_backend.py`)
2. At least one RAG collection must exist with uploaded documents
3. `RAG_DEFAULT_COLLECTION` in `config.py` must match the target collection name

## Setup and Configuration

In [ ]:
import httpx
import json
from IPython.display import display, Markdown


class LLMApiClient:
    """Unified client for the LLM API server."""

    def __init__(self, base_url: str, timeout: float = 6000.0):
        self.base_url = base_url.rstrip("/")
        self.token = None
        self.timeout = httpx.Timeout(50.0, read=timeout, write=timeout, pool=timeout)

    def _headers(self):
        return {"Authorization": f"Bearer {self.token}"} if self.token else {}

    # ---- Auth ----

    def login(self, username: str, password: str):
        r = httpx.post(
            f"{self.base_url}/api/auth/login",
            json={"username": username, "password": password},
            timeout=10.0,
        )
        r.raise_for_status()
        self.token = r.json()["access_token"]
        return r.json()

    def list_models(self):
        r = httpx.get(f"{self.base_url}/v1/models", headers=self._headers(), timeout=10.0)
        r.raise_for_status()
        return r.json()

    # ---- Chat (auto agent) ----

    def chat_new(self, model: str, user_message: str, agent_type: str = "auto"):
        """Start a new chat session. Returns (response_text, session_id)."""
        messages = [{"role": "user", "content": user_message}]
        data = {
            "model": model,
            "messages": json.dumps(messages),
            "agent_type": agent_type,
        }
        r = httpx.post(
            f"{self.base_url}/v1/chat/completions",
            data=data,
            headers=self._headers(),
            timeout=self.timeout,
        )
        r.raise_for_status()
        result = r.json()
        return result["choices"][0]["message"]["content"], result["x_session_id"]

    def chat_continue(self, model: str, session_id: str, user_message: str, agent_type: str = "auto"):
        """Continue an existing session. Returns (response_text, session_id)."""
        messages = [{"role": "user", "content": user_message}]
        data = {
            "model": model,
            "messages": json.dumps(messages),
            "session_id": session_id,
            "agent_type": agent_type,
        }
        r = httpx.post(
            f"{self.base_url}/v1/chat/completions",
            data=data,
            headers=self._headers(),
            timeout=self.timeout,
        )
        r.raise_for_status()
        result = r.json()
        return result["choices"][0]["message"]["content"], result["x_session_id"]

    # ---- RAG management (direct tools-server calls) ----

    def rag_list_collections(self, tools_base: str):
        """List all RAG collections for the authenticated user."""
        r = httpx.get(
            f"{tools_base}/api/tools/rag/collections",
            headers=self._headers(),
            timeout=10.0,
        )
        r.raise_for_status()
        return r.json()

    def rag_list_documents(self, tools_base: str, collection_name: str):
        """List documents in a RAG collection."""
        r = httpx.get(
            f"{tools_base}/api/tools/rag/collections/{collection_name}/documents",
            headers=self._headers(),
            timeout=10.0,
        )
        r.raise_for_status()
        return r.json()

    def rag_query_direct(self, tools_base: str, query: str, collection_name: str, max_results: int = 5):
        """Query RAG directly via the tools server (bypasses the agent)."""
        r = httpx.post(
            f"{tools_base}/api/tools/rag/query",
            headers=self._headers(),
            json={
                "query": query,
                "collection_name": collection_name,
                "max_results": max_results,
            },
            timeout=self.timeout,
        )
        r.raise_for_status()
        return r.json()

    def rag_upload_document(self, tools_base: str, collection_name: str, file_path: str):
        """Upload a local document to a RAG collection."""
        from pathlib import Path
        
        file_path = Path(file_path)
        if not file_path.exists():
            raise FileNotFoundError(f"File not found: {file_path}")
        
        with open(file_path, "rb") as f:
            files = {"file": (file_path.name, f, "application/octet-stream")}
            data = {"collection_name": collection_name}
            
            r = httpx.post(
                f"{tools_base}/api/tools/rag/upload",
                headers=self._headers(),
                files=files,
                data=data,
                timeout=self.timeout,
            )
        
        r.raise_for_status()
        return r.json()

    def rag_create_collection(self, tools_base: str, collection_name: str):
        """Create a new RAG collection."""
        r = httpx.post(
            f"{tools_base}/api/tools/rag/collections",
            headers=self._headers(),
            json={"collection_name": collection_name},
            timeout=10.0,
        )
        r.raise_for_status()
        return r.json()

    def rag_delete_collection(self, tools_base: str, collection_name: str):
        """Delete a RAG collection."""
        r = httpx.delete(
            f"{tools_base}/api/tools/rag/collections/{collection_name}",
            headers=self._headers(),
            timeout=10.0,
        )
        r.raise_for_status()
        return r.json()

    def rag_delete_document(self, tools_base: str, collection_name: str, document_id: str):
        """Delete a specific document from a collection."""
        r = httpx.delete(
            f"{tools_base}/api/tools/rag/collections/{collection_name}/documents/{document_id}",
            headers=self._headers(),
            timeout=10.0,
        )
        r.raise_for_status()
        return r.json()


# -------------------------------------------------------------------
# Configuration  (adjust to your environment)
# -------------------------------------------------------------------
API_BASE_URL   = "http://localhost:10007"   # Main API server
TOOLS_BASE_URL = "http://localhost:10006"   # Tools API server
USERNAME = "admin"
PASSWORD = "administrator"

client = LLMApiClient(API_BASE_URL, timeout=600.0)
print("\u2713 Client initialized")
print(f"  Main server : {API_BASE_URL}")
print(f"  Tools server: {TOOLS_BASE_URL}")

## Step 1: Authenticate and Discover Model

In [ ]:
client.login(USERNAME, PASSWORD)
models = client.list_models()
MODEL = models["data"][0]["id"]

print(f"\u2713 Logged in as: {USERNAME}")
print(f"\u2713 Using model : {MODEL}")

## Step 2: Upload Local Documents to RAG (Optional)

If you want to add your own documents to RAG, use this section. Skip to Step 3 if you already have documents uploaded.

### Supported File Formats

- **Text**: `.txt`, `.md`
- **Documents**: `.pdf`, `.docx`
- **Data**: `.json`, `.csv`, `.xlsx`, `.xls`

In [ ]:
# Create a new RAG collection and upload documents
from pathlib import Path

# Step 1: Create collection
collection_name = "default"  # Change this to your desired collection name

try:
    print(f"Creating collection '{collection_name}'...")
    result = client.rag_create_collection(TOOLS_BASE_URL, collection_name)
    
    if result.get("success"):
        print(f"✓ Collection created successfully!")
        print(f"  Collection name: {collection_name}\n")
    else:
        print(f"✗ Failed to create collection: {result.get('error')}\n")
except Exception as e:
    print(f"✗ Error creating collection: {e}\n")

# Step 2: Upload your PDF files
# Replace with your actual PDF file paths
custom_pdf_files = [
    "./USB 3.2 Revision 1.1.pdf",
    "./usb_20.pdf"
]

print(f"Uploading {len(custom_pdf_files)} documents...\n")

for pdf_file in custom_pdf_files:
    # Check if file exists
    if not Path(pdf_file).exists():
        print(f"⚠️  File not found: {pdf_file}\n")
        continue
    
    print(f"📤 Uploading: {pdf_file}")
    
    try:
        result = client.rag_upload_document(TOOLS_BASE_URL, collection_name, pdf_file)
        
        if result.get('success'):
            print(f"  ✓ Success! Chunks created: {result.get('chunks_created')}")
            print(f"  Total chunks in collection: {result.get('total_chunks')}\n")
        else:
            print(f"  ✗ Failed: {result.get('error')}\n")
    except Exception as e:
        print(f"  ✗ Error: {str(e)}\n")

## Step 3: Browse Existing RAG Collections

Let's see which collections and documents are available after any uploads.

In [ ]:
collections_result = client.rag_list_collections(TOOLS_BASE_URL)

if collections_result.get("success"):
    collections = collections_result["collections"]
    print(f"Found {len(collections)} collection(s):\n")
    for coll in collections:
        print(f"  Collection : {coll['name']}")
        print(f"  Documents  : {coll['documents']}")
        print(f"  Chunks     : {coll['chunks']}")
        print(f"  Created    : {coll['created_at']}")
        print()
else:
    print("ERROR: Could not list collections.")
    print(collections_result)

## Step 4: Inspect Documents in a Collection

Pick the collection you want to query with the RAG ReAct loop.  
Set `COLLECTION_NAME` below to match one of the collections listed above.

In [ ]:
# === Set this to the collection you want to use ===
COLLECTION_NAME = "default"

docs_result = client.rag_list_documents(TOOLS_BASE_URL, COLLECTION_NAME)

if docs_result.get("success"):
    print(f"Collection   : {docs_result['collection_name']}")
    print(f"Total docs   : {docs_result['total_documents']}")
    print(f"Total chunks : {docs_result['total_chunks']}")
    print(f"\nDocuments:")
    for doc in docs_result["documents"]:
        print(f"  - {doc['name']}  ({doc['chunks']} chunks, uploaded {doc['uploaded_at']})")
else:
    print(f"ERROR: Could not list documents in '{COLLECTION_NAME}'.")
    print(docs_result)

## Step 5: Verify RAG_DEFAULT_COLLECTION

The RAG tool uses `RAG_DEFAULT_COLLECTION` from `config.py` to decide which collection to search.  
Make sure the value in `config.py` matches the collection you want to query **before starting the servers**.

```python
# In config.py:
RAG_DEFAULT_COLLECTION = "default"  # <-- must match your target collection
```

If you need to change it, edit `config.py` and restart both servers.

In [ ]:
# Quick sanity check: read the current config value
import importlib, sys

# Add project root so we can import config
sys.path.insert(0, "..")
import config as cfg
importlib.reload(cfg)

print(f"RAG_DEFAULT_COLLECTION = \"{cfg.RAG_DEFAULT_COLLECTION}\"")
print(f"Target collection      = \"{COLLECTION_NAME}\"")

if cfg.RAG_DEFAULT_COLLECTION == COLLECTION_NAME:
    print("\n\u2713 Config matches. The auto agent will query the correct collection.")
else:
    print(f"\n\u2717 MISMATCH! Edit config.py to set RAG_DEFAULT_COLLECTION = \"{COLLECTION_NAME}\"")
    print("  Then restart both servers.")

## Step 6: RAG ReAct Loop

The `rag_react_query()` function implements a ReAct loop around direct RAG queries:

- Each iteration queries RAG, then asks the LLM to evaluate if the answer is sufficient
- The evaluation LLM uses `chat_continue` to maintain session context across iterations (it remembers what it already assessed)
- All retrieved chunks are accumulated and deduplicated across iterations
- When sufficient (or max iterations reached), all chunks are synthesized into a final comprehensive answer
- **Verbose printouts** show each step: Action, Observation, Thought

In [ ]:
import time
import hashlib
import re
from IPython.display import display, Markdown


EVALUATION_PROMPT_TEMPLATE = """You are evaluating whether a RAG (Retrieval Augmented Generation) answer sufficiently addresses the user's original question.

## Original Question
{original_query}

## Current RAG Query
{current_query}

## RAG Answer
{rag_answer}

## Retrieved Sources
{sources_summary}

## Previously Gathered Information
{accumulated_summary}

## Instructions
Evaluate whether the information gathered so far (including this iteration) adequately answers the original question with specific, accurate, and detailed information.

Consider:
1. Does the answer contain concrete data (numbers, specs, definitions) rather than vague generalizations?
2. Are there obvious gaps or missing aspects of the question that haven't been addressed?
3. Would a different search query likely retrieve additional useful information?

Respond in EXACTLY this format (no extra text before or after):
sufficient: true OR sufficient: false
reasoning: <1-2 sentences explaining your judgment>
follow_up_query: <a refined search query to fill gaps, or "none" if sufficient>"""


SYNTHESIS_PROMPT_TEMPLATE = """Based on the following retrieved document chunks, provide a comprehensive and detailed answer to the user's question.

## User's Question
{original_query}

## Retrieved Document Chunks
{all_chunks_formatted}

## Instructions
- Synthesize ALL the retrieved chunks into a single, well-organized answer
- Include specific data, numbers, and technical details from the documents
- If chunks contain contradictory information, note the discrepancy
- Structure your answer clearly with sections/bullet points as appropriate
- Only use information from the provided chunks — do not add external knowledge
- If the chunks don't fully answer the question, clearly state what information is missing"""


def rag_react_query(query, collection_name, model, max_iterations=3, max_results=5):
    """
    ReAct loop for RAG queries.
    
    Iteratively queries RAG, evaluates sufficiency via LLM, and refines
    the search query until the answer is comprehensive.
    
    Args:
        query: Original user question
        collection_name: RAG collection to search
        model: LLM model name for evaluation/synthesis
        max_iterations: Maximum number of retrieval iterations (default 3)
        max_results: Number of chunks per RAG query (default 5)
    
    Returns:
        (final_answer, history) tuple
    """
    all_chunks = []
    seen_hashes = set()
    history = []
    current_query = query
    eval_session_id = None
    total_start = time.time()

    print(f"{'=' * 80}")
    print(f"  RAG ReAct Loop")
    print(f"  Original Question: {query}")
    print(f"  Collection: {collection_name} | Max Iterations: {max_iterations}")
    print(f"{'=' * 80}\n")

    for iteration in range(1, max_iterations + 1):
        iter_start = time.time()
        print(f"--- Iteration {iteration}/{max_iterations} {'---' * 20}")

        # ========== ACTION: RAG Retrieval ==========
        print(f"\n  [ACTION] Querying RAG...")
        print(f"    Query: \"{current_query}\"")

        rag_start = time.time()
        result = client.rag_query_direct(
            TOOLS_BASE_URL,
            query=current_query,
            collection_name=collection_name,
            max_results=max_results,
        )
        rag_time = time.time() - rag_start

        if not result.get("success"):
            print(f"    ERROR: {result.get('error')}")
            history.append({
                "iteration": iteration,
                "query": current_query,
                "error": result.get("error"),
            })
            break

        rag_answer = result["answer"]
        data = result.get("data", {})
        documents = data.get("documents", [])

        print(f"    Retrieved: {len(documents)} chunks in {rag_time:.2f}s")

        # ========== OBSERVATION: Show retrieved results ==========
        print(f"\n  [OBSERVATION] Retrieved chunks:")
        new_chunks_count = 0
        sources_lines = []
        for i, doc in enumerate(documents, 1):
            score = doc.get("score", 0)
            source_line = f"    [{i}] {doc['document']} chunk {doc['chunk_index']} (score: {score:.3f})"
            print(source_line)
            sources_lines.append(source_line)

            # Deduplicate by content hash
            chunk_hash = hashlib.md5(doc["chunk"].encode()).hexdigest()
            if chunk_hash not in seen_hashes:
                seen_hashes.add(chunk_hash)
                all_chunks.append(doc)
                new_chunks_count += 1

        print(f"\n    New unique chunks: {new_chunks_count} | Total accumulated: {len(all_chunks)}")
        print(f"\n  [RAG ANSWER] (truncated to 300 chars):")
        print(f"    {rag_answer[:300]}{'...' if len(rag_answer) > 300 else ''}")

        # ========== THOUGHT: LLM evaluates sufficiency ==========
        print(f"\n  [THOUGHT] Evaluating answer sufficiency via LLM...")

        accumulated_summary = "None (first iteration)" if iteration == 1 else \
            f"{len(all_chunks)} total chunks accumulated from {iteration - 1} previous queries"

        eval_prompt = EVALUATION_PROMPT_TEMPLATE.format(
            original_query=query,
            current_query=current_query,
            rag_answer=rag_answer,
            sources_summary="\n".join(sources_lines),
            accumulated_summary=accumulated_summary,
        )

        eval_start = time.time()
        if eval_session_id is None:
            eval_response, eval_session_id = client.chat_new(
                model=model,
                user_message=eval_prompt,
                agent_type="chat",
            )
        else:
            eval_response, eval_session_id = client.chat_continue(
                model=model,
                session_id=eval_session_id,
                user_message=eval_prompt,
                agent_type="chat",
            )
        eval_time = time.time() - eval_start

        # Parse evaluation response
        is_sufficient = False
        reasoning = ""
        follow_up_query = ""

        for line in eval_response.strip().split("\n"):
            line_lower = line.strip().lower()
            if line_lower.startswith("sufficient:"):
                is_sufficient = "true" in line_lower
            elif line_lower.startswith("reasoning:"):
                reasoning = line.strip()[len("reasoning:"):].strip()
            elif line_lower.startswith("follow_up_query:"):
                follow_up_query = line.strip()[len("follow_up_query:"):].strip()

        iter_time = time.time() - iter_start

        print(f"    Sufficient: {'YES' if is_sufficient else 'NO'}")
        print(f"    Reasoning: {reasoning}")
        if not is_sufficient and follow_up_query and follow_up_query.lower() != "none":
            print(f"    Follow-up query: \"{follow_up_query}\"")
        print(f"    (eval LLM: {eval_time:.2f}s | iteration total: {iter_time:.2f}s)")

        history.append({
            "iteration": iteration,
            "query": current_query,
            "rag_answer": rag_answer,
            "chunks_retrieved": len(documents),
            "new_unique_chunks": new_chunks_count,
            "sufficient": is_sufficient,
            "reasoning": reasoning,
            "follow_up_query": follow_up_query,
            "rag_time": rag_time,
            "eval_time": eval_time,
        })

        if is_sufficient:
            print(f"\n  >>> Answer deemed SUFFICIENT at iteration {iteration}. Proceeding to synthesis.")
            break

        if follow_up_query and follow_up_query.lower() != "none":
            current_query = follow_up_query
        else:
            print(f"\n  >>> No follow-up query generated. Stopping loop.")
            break

        print()

    # ========== FINAL SYNTHESIS ==========
    print(f"\n{'=' * 80}")
    print(f"  [SYNTHESIS] Generating final answer from {len(all_chunks)} accumulated chunks...")
    print(f"{'=' * 80}")

    chunks_formatted = "\n\n".join([
        f"[Chunk {i+1}] Source: {doc['document']}, Chunk {doc['chunk_index']} "
        f"(Score: {doc.get('score', 0):.3f}):\n{doc['chunk']}"
        for i, doc in enumerate(all_chunks)
    ])

    synthesis_prompt = SYNTHESIS_PROMPT_TEMPLATE.format(
        original_query=query,
        all_chunks_formatted=chunks_formatted,
    )

    synth_start = time.time()
    final_answer, _ = client.chat_new(
        model=model,
        user_message=synthesis_prompt,
        agent_type="chat",
    )
    synth_time = time.time() - synth_start
    total_time = time.time() - total_start

    # Display final answer
    print(f"\n  Synthesis LLM time: {synth_time:.2f}s")
    print(f"  Total elapsed: {total_time:.2f}s")
    print(f"  Iterations used: {len(history)}/{max_iterations}")
    print(f"  Total unique chunks: {len(all_chunks)}")
    print(f"\n{'=' * 80}\n")

    display(Markdown(f"### Final Answer\n\n{final_answer}"))

    # Display iteration summary
    summary_lines = ["### ReAct Loop Summary\n", "| Iter | Query | Chunks | New | Sufficient | Reasoning |",
                     "|------|-------|--------|-----|------------|-----------|"]
    for h in history:
        q = h['query'][:40] + "..." if len(h['query']) > 40 else h['query']
        summary_lines.append(
            f"| {h['iteration']} | {q} | {h.get('chunks_retrieved', 0)} | "
            f"{h.get('new_unique_chunks', 0)} | {'Yes' if h.get('sufficient') else 'No'} | "
            f"{h.get('reasoning', '')[:60]} |"
        )
    summary_lines.append(f"\n**Total time:** {total_time:.2f}s | **Total chunks:** {len(all_chunks)}")
    display(Markdown("\n".join(summary_lines)))

    return final_answer, history


print("ReAct RAG loop utility loaded")

## Question 1: C-PHY Insertion Loss at 3.9Gsps

In [ ]:
query_1 = "C-PHY가 3.9Gsps로 동작할 때 Insertion Loss 스펙을 알려줘"
rag_react_query(query_1, COLLECTION_NAME, MODEL)

## Question 2: MIPI C-PHY SKEW Error

In [ ]:
query_2 = "MIPI C-PHY에서 SKEW Error에 대해 알려줘"
rag_react_query(query_2, COLLECTION_NAME, MODEL)

## Question 3: MIPI M-PHY v6.0 FOM and EOM

In [ ]:
query_3 = "MIPI M-PHY v6.0의 FOM과 EOM에 대해서 알려줘. 각각에 대해 비교해주고, Spec.에서는 Mandatory사항인지, Option사항인지 확인해줘"
rag_react_query(query_3, COLLECTION_NAME, MODEL)

## Question 4: USB3.2 LTSSM and RX.Detect

In [ ]:
query_4 = "USB3.2의 LTSSM에 대해서 자세히 설명해주고, 특히 RX.Detect에 대해서 자세히 설명해줘"
rag_react_query(query_4, COLLECTION_NAME, MODEL)

## Question 5: PCIe AER (Advanced Error Reporting)

In [ ]:
query_5 = "PCIe AER에 대해서 설명해주고, 대표적인 Error 들에 대해서 비교 설명해줘"
rag_react_query(query_5, COLLECTION_NAME, MODEL)

## Question 6: USB3.2 Gen1/Gen2 Transmitter Eye Mask

In [ ]:
query_6 = "USB3.2의 Gen1(5GT/s), Gen2(10GT/s)의 Transmitter eye mask 규격에 대해서 측정 위치와 Eye Height spec.(Minimum, Maximum)에 대해서 비교 설명해줘"
rag_react_query(query_6, COLLECTION_NAME, MODEL)

## Question 7: DisplayPort Link Training

In [ ]:
query_7 = "DP Source와 Sink간 Link training 절차에 대해서 설명해주고, Clock Recovery, Channel equalization, Symbol Lock을 수행하는 프로세스에 대해서 알려줘"
rag_react_query(query_7, COLLECTION_NAME, MODEL)

## Question 8: USB2.0 High-Speed Disconnect Level (VHSDSC)

In [ ]:
query_8 = "USB2.0의 high-speed Disconnect level(VHSDSC)에 대해서 설명해주고, 최초 USB2.0 spec.대비 USB 2.0 DCR ECN에 의해 변경된 사항에 대해 알려줘"
rag_react_query(query_8, COLLECTION_NAME, MODEL)

## Question 9: USB4.0, USB3.2, USB2.0 Comparison

In [ ]:
query_9 = "USB4.0, USB3.2, USB2.0 각각의 Spec.에 대해서 신호를 전송하는 전송방식과 최대 동작속도와 물리적인 배선의 차이점에 대해서 설명해줘"
rag_react_query(query_9, COLLECTION_NAME, MODEL)

## Question 10: UFS, Unipro AFCx Frame Transmission

In [ ]:
query_10 = "UFS, Unipro의 AFCx Frame Transmission에 대한 설명을 해주고, FCx_PROTECTION_TIMER or TCx_REPLAY_TIMER에 대해서도 설명해줘"
rag_react_query(query_10, COLLECTION_NAME, MODEL)

## Question 11: UFS, MIPI MPHY, Unipro Link Initialization

In [ ]:
query_11 = "UFS는 MIPI MPHY, Unipro로 이루어지는데 각각에 대해서 설명을 해주고, Link initialization process에 대해서 확인해줘"
rag_react_query(query_11, COLLECTION_NAME, MODEL)

## Question 12: DisplayPort 1.4 Voltage Swing and Pre-emphasis

In [ ]:
query_12 = "DP1.4 spec.에서 정의하는 Voltage Swing and Pre-emphasis에 대해서 설명해주고, Voltage swing level과 TX Emphasis Level이 0/1/2/3으로 설정되는 이유에 대해서 알려줘"
rag_react_query(query_12, COLLECTION_NAME, MODEL)

## Question 13: MIPI C-PHY Crosstalk Spec

In [ ]:
query_13 = "MIPI C-PHY crosstalk spec은?"
rag_react_query(query_13, COLLECTION_NAME, MODEL)

## Question 14: Interface Impedance Guide

In [ ]:
query_14 = "인터페이스별 임피던스 가이드를 알려줘"
rag_react_query(query_14, COLLECTION_NAME, MODEL)

## Question 15: MIPI D-PHY Initial vs Periodic Skew Calibration

In [ ]:
query_15 = "MIPI D-phy에서 Initial Skew cal과 Periodic Skew cal의 차이에 대해서 설명해줘"
rag_react_query(query_15, COLLECTION_NAME, MODEL)

## Question 16: USB3.2 Gen2 Electrical Compliance Test

In [ ]:
query_16 = "USB3.2 Gen2 동작 시의 Electrical Compliance Test 항목에 대해서 알려줘"
rag_react_query(query_16, COLLECTION_NAME, MODEL)

## Question 17: eUSB Repeater Mode eSE1 Signal

In [ ]:
query_17 = "eUSB의 Repeater mode에서 eSE1 신호가 발생하는 경우에 대해 설명해 주세요"
rag_react_query(query_17, COLLECTION_NAME, MODEL)

## Question 18: USB vs eUSB FS Signaling Difference

In [ ]:
query_18 = "USB와 eUSB의 FS signaling 차이를 설명해 주세요"
rag_react_query(query_18, COLLECTION_NAME, MODEL)

## Question 19: USB3.0 Voltage Level

In [ ]:
query_19 = "USB3.0 전압 레벨은 어떻게되지?"
rag_react_query(query_19, COLLECTION_NAME, MODEL)

## Question 20: USB2.0 EQ Impact on Design Length

In [ ]:
query_20 = "USB2.0에서 EQ을 사용한 경우와 사용하지 않은 경우에 설계 길이의 차이는 얼마나 될까?"
rag_react_query(query_20, COLLECTION_NAME, MODEL)

## Question 21: USB3.2 Gen2x2 8b/10b Encoding Maximum Speed

In [ ]:
query_21 = "USB3.2 Gen2x2 에서 8b/10b 인코딩 방식을 사용할때 최대 전송 속도는 얼마가 될까?"
rag_react_query(query_21, COLLECTION_NAME, MODEL)

## Summary

This notebook demonstrated:

1. **Uploading local documents to RAG** — Add your own documents (PDF, TXT, MD, DOCX, CSV, etc.) to RAG collections
2. **Browsing existing RAG collections** — List collections and inspect their documents
3. **Config verification** — Ensure `RAG_DEFAULT_COLLECTION` points to the right collection
4. **RAG ReAct Loop** — Iterative retrieval with LLM-based sufficiency evaluation
5. **Accumulated retrieval** — Multiple RAG queries build up a comprehensive chunk set, deduplicated by content
6. **LLM evaluation with session memory** — `chat_continue` allows the evaluator LLM to remember previous assessments across iterations
7. **Final synthesis** — All accumulated chunks are synthesized into a comprehensive answer
8. **Collection management** — Create/delete collections and remove specific documents

### ReAct Loop Flow

```
rag_react_query(question)
  ├── Iteration 1: RAG query (original) → LLM evaluates → sufficient? 
  │     ├── Yes → Synthesize from all chunks → Done
  │     └── No  → Generate follow-up query
  ├── Iteration 2: RAG query (refined) → LLM evaluates (with session memory) → sufficient?
  │     └── ...
  └── Final: Synthesize all accumulated unique chunks → Comprehensive answer
```

### API Endpoints Used

| Endpoint | Purpose |
|----------|--------|
| `POST /api/auth/login` | Authentication |
| `GET /v1/models` | Discover available models |
| `POST /api/tools/rag/upload` | Upload document to RAG collection |
| `POST /api/tools/rag/collections` | Create new collection |
| `GET /api/tools/rag/collections` | List existing collections |
| `GET /api/tools/rag/collections/{name}/documents` | List documents in a collection |
| `DELETE /api/tools/rag/collections/{name}/documents/{id}` | Delete specific document |
| `DELETE /api/tools/rag/collections/{name}` | Delete entire collection |
| `POST /api/tools/rag/query` | Direct RAG query (tools server) |
| `POST /v1/chat/completions` | LLM evaluation & synthesis (main server, chat agent) |

## Step 7: Collection Management (Optional)

You can also create new collections, delete collections, or delete specific documents.

In [ ]:
# Create a new collection
# Uncomment to create

# new_collection_name = "my_new_collection"
# try:
#     result = client.rag_create_collection(TOOLS_BASE_URL, new_collection_name)
#     if result.get("success"):
#         print(f"✓ Collection '{new_collection_name}' created successfully")
#     else:
#         print(f"✗ Failed: {result.get('error')}")
# except Exception as e:
#     print(f"✗ Error: {e}")

In [ ]:
# Delete a document from a collection
# First, list documents to get the document ID, then uncomment to delete

# collection_name = "default"
# document_id = "abc123..."  # Get this from list_documents result

# try:
#     result = client.rag_delete_document(TOOLS_BASE_URL, collection_name, document_id)
#     if result.get("success"):
#         print(f"✓ Document deleted successfully")
#         print(f"  Deleted: {result.get('deleted_document')}")
#         print(f"  Remaining documents: {result.get('remaining_documents')}")
#     else:
#         print(f"✗ Failed: {result.get('error')}")
# except Exception as e:
#     print(f"✗ Error: {e}")

In [ ]:
# Delete an entire collection (use with caution!)
# Uncomment to delete

# collection_to_delete = "test_collection"
# try:
#     result = client.rag_delete_collection(TOOLS_BASE_URL, collection_to_delete)
#     if result.get("success"):
#         print(f"✓ Collection '{collection_to_delete}' deleted successfully")
#     else:
#         print(f"✗ Failed: {result.get('error')}")
# except Exception as e:
#     print(f"✗ Error: {e}")